In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import email as em

from datetime import date, timedelta, datetime
from time import mktime
from email.utils import parsedate, parsedate_tz
from email.parser import HeaderParser

from classes.gmail import GmailAccount

In [3]:
password = open(os.path.expanduser('~/.ssh/pw'), 'r').read().strip()

In [117]:
tyler = GmailAccount(username='tyleha@gmail.com', password=password)
tyler.login()

('OK', [b'tyleha@gmail.com authenticated (Success)'])

In [118]:
daysback = 60
notsince = 0
since = (date.today() - timedelta(daysback)).strftime("%d-%b-%Y")
before = (date.today() - timedelta(notsince)).strftime("%d-%b-%Y")

SEARCH = '(SENTSINCE {si} SENTBEFORE {bf})'.format(si=since, bf=before)
BODY = '(BODY.PEEK[TEXT])'
ALL_HEADERS = '(BODY.PEEK[HEADER.FIELDS (DATE TO CC FROM SUBJECT)])'
DATE = '(BODY.PEEK[HEADER.FIELDS (DATE)])'

In [119]:
#LOAD GMAIL EMAILS
received = tyler.load_parse_query(SEARCH, ALL_HEADERS, 'INBOX')
# sent = tyler.load_parse_query(SEARCH, ALL_HEADERS, '[Gmail]/Sent Mail')

# Do the pandas thing

In [42]:
import pandas as pd
import numpy as np

In [133]:
df = pd.DataFrame([dict(email._headers) for email in received])

# df['timestamp'] = df.Date.map(pd.Timestamp)
# df['datetime'] = df.Date.apply(lambda x: pd.Timestamp(x).to_datetime())

df['datetime'] = df['Date'].apply(lambda x: datetime.fromtimestamp(mktime(parsedate(x))))
df['timestamp'] = df.datetime.map(pd.Timestamp)
df['period'] = df.timestamp.apply(lambda x: x.to_period(freq='M'))

# df['date_group'] = df['datetime'].apply(lambda x: x.strftime('%Y-%b'))
df['hour'] = df.timestamp.map(lambda x: x.hour)

In [152]:
# Scale our heatmap to the min/max range of our email data.
mindate = df.datetime.min()
maxdate = df.datetime.max()
pr = pd.period_range(mindate, maxdate, freq='M')

# Initialize a new HeatMap dataframe where the indicies are actually Periods of time!
# Size the frame anticipating the correct number of rows (periods) and columns (hours in a day)
hm = pd.DataFrame(np.zeros([len(pr), 24]) , index=pr)

for period in pr:
    # HERE'S where the magic happens...with pandas, when you structure your data correctly, it can be so terse that
    # you almost aren't sure the program does what it says it does...
    # For this period (month), find all emails within this month and count how many emails were received
    # within each hour of the day in that month. Wow. Takes more words to explain than to code.
    # Normally that would take you 3 or 4 for loops, constantly trying to catch edge cases and fencepost
    # problems. But here, it's as simple as you please.
    hm.ix[period] = df[df.period == period].hour.value_counts()
    
# If for some weird reason there was ever an hour period where you received no email,
# fill those NaNs with zeros.
hm.fillna(0, inplace=True)

In [154]:
hm

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
2015-11,6,8,6,7,5,2,12,9,15,22,...,17,20,16,20,14,18,34,18,12,13
2015-12,6,7,18,14,3,7,10,13,20,17,...,24,31,31,23,22,23,17,37,16,14


# Scratch City